In [ ]:
%%writefile task1.cpp

#include<bits/stdc++.h> // Подключаем все стандартные библиотеки

#include <bits/stdc++.h>
#include <omp.h>
using namespace std;
using namespace chrono;

/* =========================
   Заполнение массива
   ========================= */
void fillArray(vector<int>& a) {
    for (int& x : a)
        x = rand() % 100000;
}

/* =========================
   ПОСЛЕДОВАТЕЛЬНЫЕ СОРТИРОВКИ
   ========================= */

// Пузырьком
void bubbleSortSeq(vector<int>& a) {
    int n = a.size();
    for (int i = 0; i < n - 1; i++)
        for (int j = 0; j < n - i - 1; j++)
            if (a[j] > a[j + 1])
                swap(a[j], a[j + 1]);
}

// Выбором
void selectionSortSeq(vector<int>& a) {
    int n = a.size();
    for (int i = 0; i < n - 1; i++) {
        int minIdx = i;
        for (int j = i + 1; j < n; j++)
            if (a[j] < a[minIdx])
                minIdx = j;
        swap(a[i], a[minIdx]);
    }
}

// Вставками
void insertionSortSeq(vector<int>& a) {
    int n = a.size();
    for (int i = 1; i < n; i++) {
        int key = a[i];
        int j = i - 1;
        while (j >= 0 && a[j] > key) {
            a[j + 1] = a[j];
            j--;
        }
        a[j + 1] = key;
    }
}

/* =========================
   ПАРАЛЛЕЛЬНЫЕ (OpenMP)
   ========================= */

// Пузырёк (odd-even sort)
void bubbleSortPar(vector<int>& a) {
    int n = a.size();
    for (int phase = 0; phase < n; phase++) {
        #pragma omp parallel for
        for (int i = phase % 2; i < n - 1; i += 2)
            if (a[i] > a[i + 1])
                swap(a[i], a[i + 1]);
    }
}

// Выбор (параллельный поиск минимума)
void selectionSortPar(vector<int>& a) {
    int n = a.size();
    for (int i = 0; i < n - 1; i++) {
        int minIdx = i;

        #pragma omp parallel for
        for (int j = i + 1; j < n; j++) {
            if (a[j] < a[minIdx]) {
                #pragma omp critical
                {
                    if (a[j] < a[minIdx])
                        minIdx = j;
                }
            }
        }
        swap(a[i], a[minIdx]);
    }
}

// Вставка — оставляем последовательной
void insertionSortPar(vector<int>& a) {
    insertionSortSeq(a);
}

/* =========================
   Замер времени
   ========================= */
void test(const string& name,
          void (*sortFunc)(vector<int>&),
          vector<int> a) {

    auto start = high_resolution_clock::now();
    sortFunc(a);
    auto end = high_resolution_clock::now();

    cout << name << ": "
         << duration<double>(end - start).count()
         << " сек\n";
}

int main() {
    srand(time(0));

    vector<int> sizes = {1000, 10000, 100000};

    for (int n : sizes) {
        cout << "\nРазмер массива: " << n << endl;
        vector<int> a(n);

        fillArray(a); test("Bubble (seq)", bubbleSortSeq, a);
        fillArray(a); test("Bubble (omp)", bubbleSortPar, a);

        fillArray(a); test("Selection (seq)", selectionSortSeq, a);
        fillArray(a); test("Selection (omp)", selectionSortPar, a);

        fillArray(a); test("Insertion (seq)", insertionSortSeq, a);
        fillArray(a); test("Insertion (omp)", insertionSortPar, a);
    }
    return 0;
}


Writing task1.cpp


In [ ]:
! g++ -fopenmp -o2 task1.cpp -o anyname
!./anyname


Размер массива: 1000
Bubble (seq): 0.00469133 сек
Bubble (omp): 0.00485464 сек
Selection (seq): 0.00295903 сек
Selection (omp): 0.002707 сек
Insertion (seq): 0.00235132 сек
Insertion (omp): 0.00220313 сек

Размер массива: 10000
Bubble (seq): 0.472076 сек
Bubble (omp): 0.483553 сек
Selection (seq): 0.210249 сек
Selection (omp): 0.28052 сек
Insertion (seq): 0.157654 сек
Insertion (omp): 0.165234 сек

Размер массива: 100000
Bubble (seq): 65.5169 сек
Bubble (omp): 52.4078 сек
Selection (seq): 22.1049 сек
Selection (omp): 24.8236 сек
Insertion (seq): 16.6145 сек
Insertion (omp): 15.9162 сек
